In [1]:
main_dir='../'

all_file=main_dir+'features/white_filter.txt'

import os
import sys
tools_dir='/home/hr/TensorFlow-Tutorials-master/tools/'
sys.path.append(tools_dir)
import everything as et

#main_dir='/home/hr/TensorFlow-Tutorials-master/myPros/'
work_dir=main_dir+'pifu_features_cnn/'
all_dir=main_dir+'256_patch/'

label_size=2

pixel_len=256
color_channel=3

_=et.construct_cross_validate(all_dir=all_dir,all_file=all_file,work_dir=work_dir)

i=0
cross_dir=os.path.join(work_dir,'x_validate')
tr_fn=os.path.join(cross_dir,'train'+str(i)+'.txt')
te_fn=os.path.join(cross_dir,'test'+str(i)+'.txt')

trX,trY,teX,teY=et.input_data_by_fname(all_dir, tr_fn, te_fn, pixel_len, color_channel, label_size)

import tensorflow as tf
import tensorflow.contrib.slim as slim
import vgg as inception_v1

BATCH_SIZE=50
start_lr=0.0001
stepsize=10000
decay_ratio=0.5

inputs=tf.placeholder("float", shape=[BATCH_SIZE, pixel_len, pixel_len, color_channel], name='inputs')
labels=tf.placeholder(dtype=tf.int16, shape=[BATCH_SIZE, label_size], name='labels')

global_step = tf.Variable(0, trainable=False)
learning_rate = tf.train.exponential_decay(start_lr, global_step,
                                     stepsize, decay_ratio, staircase=True)

p_keep_hidden = tf.placeholder("float")

In [2]:
with slim.arg_scope(inception_v1.vgg_arg_scope(weight_decay=0.0005)):
    output, end_points=inception_v1.vgg_a(inputs=inputs, dropout_keep_prob=p_keep_hidden, num_classes=label_size)

In [3]:
prediction = slim.softmax(output)
class_loss = slim.losses.softmax_cross_entropy(prediction, labels) 
total_loss = slim.losses.get_total_loss()

train_op = tf.train.MomentumOptimizer(learning_rate, 0.9).minimize(total_loss, global_step=global_step)
predict_op = tf.argmax(prediction,1)
correct=tf.equal(predict_op,tf.argmax(labels,1))
accuracy=tf.reduce_mean(tf.cast(correct, "float"))

train_loss_sum=tf.scalar_summary('train_loss', total_loss)
test_loss_sum=tf.scalar_summary('test_loss', total_loss)
train_acc_sum=tf.scalar_summary('train_accuracy', accuracy)
test_acc_sum=tf.scalar_summary('test_accuracy', accuracy)

In [4]:
def shuffle_examples(X, Y):
    indices=np.arange(X.shape[0])
    np.random.shuffle(indices)
    
    sX=np.zeros_like(X)
    sY=np.zeros_like(Y)
    #sL=np.zeros_like(L)
    for i in range(len(indices)):
        sX[i]=X[indices[i]]
        sY[i]=Y[indices[i]]
    #    sL[i]=L[indices[i]]
        
    return sX, sY

import numpy as np
def do_mirror(trX, trY):
    col_num=trX.shape[0]
    mirror_x=np.zeros_like(trX)
    mirror_y=np.zeros_like(trY)
    temp=np.zeros_like(mirror_x[0])
    for i in range(col_num):
        img=trX[i].copy()
        for c in range(img.shape[0]/2):
            temp=img[c,:,:].copy()
            img[c,:,:]=img[-c-1,:,:].copy()
            img[-c-1,:,:]=temp.copy()
        mirror_x[i]=img.copy()
        mirror_y[i]=trY[i].copy()
    
    trX=np.insert(trX, trX.shape[0], mirror_x, axis=0)
    trY=np.insert(trY, trY.shape[0], mirror_y, axis=0)
        
    return trX, trY

def cal_mean(trX):
    mean_image=np.zeros([pixel_len,pixel_len,color_channel], dtype=np.float16)
    batch_num=200
    temp_mean=np.zeros([np.ceil(trX.shape[0]*1.0/batch_num)])
    temp_batch=np.zeros_like(temp_mean)
    
    for channel in range(trX.shape[3]):
        for i in range(mean_image.shape[0]):
            for j in range(mean_image.shape[1]):
                batch_id=0

                t=0
                for s, t in zip(range(0, trX.shape[0], batch_num), range(batch_num, trX.shape[0]+1, batch_num)):
                    temp_batch[batch_id]=batch_num
                    temp_mean[batch_id]=np.mean(trX[s:t,i,j,channel], axis=0)
                    batch_id+=1
                if trX.shape[0]>t:
                    temp_batch[batch_id]=trX.shape[0]-t
                    temp_mean[batch_id]=np.mean(trX[t:,i,j,channel])
                for k in range(temp_batch.shape[0]):
                    temp_batch[k]=temp_batch[k]/trX.shape[0]
                    mean_image[i,j,channel]+=temp_batch[k]*temp_mean[k]
    
    return mean_image

def minus_mean(X, mean_image):
    for channel in range(X.shape[3]):
        for elm in range(X.shape[0]):
            X[elm, :, :, channel]-=mean_image[:,:,channel]

In [5]:
i = 0
max_iternum=15000
log_dir='log'+'/'
model_dir='models'+'/'
mean_files_dir='mean_files'+'/'
    
#te_fn=work_dir+'x_validate/test'+str(i)+'.txt'
#tr_fn=work_dir+'x_validate/train'+str(i)+'.txt'

#trX, trY, teX, teY=et.input_data(all_dir, tr_fn, te_fn,pixel_len,color_channel,label_size)
print 'input_data() done!'
trX, trY=do_mirror(trX=trX, trY=trY)

print 'do_mirror() done!'
mean_image=cal_mean(trX)
print 'cal_mean() done!'
minus_mean(trX, mean_image=mean_image)
minus_mean(teX, mean_image=mean_image)
print 'minus_mean() done!'
print 'preprocess done!'

mean_i=mean_files_dir+'mean_'+str(i)+'.npy'
np.save(mean_i, mean_image)

saver=tf.train.Saver()

test_size=BATCH_SIZE
train_it=0

with tf.Session() as sess:
    log_i=log_dir+str(i)+'/'
    if os.path.exists(log_i)==False:
        os.mkdir(log_i)
    summary_writer = tf.train.SummaryWriter(log_i, sess.graph)

    tf.initialize_all_variables().run()

    while train_it<max_iternum:
        training_batch = zip(range(0, len(trX), BATCH_SIZE),
                             range(BATCH_SIZE, len(trX)+1, BATCH_SIZE))
        for start, end in training_batch:
            sess.run(train_op, feed_dict={inputs: trX[start:end], labels: trY[start:end], p_keep_hidden:0.5})
            if train_it%20==0:
                print sess.run(learning_rate)
                train_loss,summary = sess.run([total_loss, train_loss_sum], 
                                       feed_dict={inputs:trX[start:end], 
                                                  labels:trY[start:end], 
                                                  p_keep_hidden: 0.5})
                summary_writer.add_summary(summary, train_it)
                print train_it, train_loss,
                train_accuracy,summary = sess.run([accuracy, train_acc_sum],
                                                      feed_dict={inputs:trX[start:end], 
                                                                 labels:trY[start:end], 
                                                                 p_keep_hidden: 0.5})
                summary_writer.add_summary(summary, train_it)
                print train_accuracy

                test_indices = np.arange(len(teX)) # Get A Test Batch
                np.random.shuffle(test_indices)
                test_indices = test_indices[0:test_size]

                test_loss, summary=sess.run([total_loss, test_loss_sum],
                                           feed_dict={inputs:teX[test_indices],
                                                     labels:teY[test_indices],
                                                     p_keep_hidden:1.0})
                summary_writer.add_summary(summary, train_it)
                print train_it, test_loss,
                test_accuracy, summary=sess.run([accuracy, test_acc_sum],
                                               feed_dict={inputs:teX[test_indices],
                                                         labels:teY[test_indices],
                                                         p_keep_hidden:1.0})
                summary_writer.add_summary(summary, train_it)
                print test_accuracy
            train_it+=1
        trX, trY=shuffle_examples(trX, trY)

    save_path=model_dir+str(i)+'/'
    if os.path.exists(save_path)==False:
        os.mkdir(save_path)
    saved_path = saver.save(sess, save_path+'vgg_a_white_filter.ckpt')
    print(saved_path)
    i+=1

input_data() done!
do_mirror() done!


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:38: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future


cal_mean() done!
minus_mean() done!
preprocess done!
0.0001
0 2.56449 0.54
0 2.53758 0.64
0.0001
20 2.50775 0.66
20 2.5336 0.5
0.0001
40 2.44315 0.7
40 2.50432 0.58
0.0001
60 2.46389 0.68
60 2.54017 0.46
0.0001
80 2.41092 0.74
80 2.43289 0.62
0.0001
100 2.39789 0.8
100 2.39957 0.72
0.0001
120 2.40842 0.76
120 2.48136 0.62
0.0001
140 2.35658 0.88
140 2.40692 0.72
0.0001
160 2.37064 0.76
160 2.48954 0.62
0.0001
180 2.35245 0.84
180 2.45462 0.64
0.0001
200 2.34783 0.84
200 2.48982 0.6
0.0001
220 2.31915 0.82
220 2.55751 0.52
0.0001
240 2.35991 0.8
240 2.51514 0.6
0.0001
260 2.30027 0.82
260 2.47573 0.64
0.0001
280 2.29375 0.9
280 2.51018 0.66
0.0001
300 2.25732 0.94
300 2.46022 0.68
0.0001
320 2.34361 0.82
320 2.52603 0.62
0.0001
340 2.29178 0.86
340 2.41934 0.68
0.0001
360 2.32183 0.86
360 2.45432 0.7
0.0001
380 2.31275 0.86
380 2.38079 0.74
0.0001
400 2.3904 0.76
400 2.50747 0.62
0.0001
420 2.34276 0.8
420 2.35018 0.82
0.0001
440 2.30312 0.86
440 2.52734 0.6
0.0001
460 2.34573 0.82
460 